# Lab5: Classifying real-world images

In last exercise we built a CNN to classify MNIST images. 
- We also used a callback to cancel training if a certain accuracy is reached.
- We reshaped our training and test images in order to pass them to convolution layer.

We improved the performance of classification by using convolutions which spotted features in the image. The rest of the network matched those features to the labels instead of using raw pixels and hoping for the best.

This technique in theory should work for images that are more complex than the fashion-mnist and mnist dataset. Here we will see if we can apply the same technique to other images. \
_(Images in which subject is not centered and facing in the same direction)_

For this we will use **horses_and_humans dataset**. 

In [23]:
import os
dataset_path = "D:\\local_ws\\datasets\\horse-or-human\\"

# specify directories:
train_horse_dir = os.path.join(dataset_path + 'train\\horses')
train_human_dir = os.path.join(dataset_path + 'train\\humans')
validation_horse_dir = os.path.join(dataset_path + 'validation\\horses')
validation_human_dir = os.path.join(dataset_path + 'validation\\humans')

# check how the filenames look like in these directories
train_horse_names = os.listdir(train_horse_dir); print(train_horse_names[:10])
train_human_names = os.listdir(train_human_dir); print(train_human_names[:10])

# the total number of horse and human images in the directories
print('total training horse images: ', len(train_horse_names))
print('total training human images: ', len(train_human_names))

['horse01-0.png', 'horse01-1.png', 'horse01-2.png', 'horse01-3.png', 'horse01-4.png', 'horse01-5.png', 'horse01-6.png', 'horse01-7.png', 'horse01-8.png', 'horse01-9.png']
['human01-00.png', 'human01-01.png', 'human01-02.png', 'human01-03.png', 'human01-04.png', 'human01-05.png', 'human01-06.png', 'human01-07.png', 'human01-08.png', 'human01-09.png']
total training horse images:  500
total training human images:  527


In [24]:
# Lets take a look at the pictures

# First configuring matplotlib parameters
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Params of our graph, we will output these images in 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

In [ ]:
# Displaying a batch of 8 horse and 8 human pictures.
# Rerun this cell to see a fresh batch each time





### Loading the dataset

As we will not be loading the dataset through inbuilt function, we will need an easy way to load the dataset.

**Tensorflow supports the use of sub-directories.**
- If I have a master directory which contains two sub-directories called Training and Validation.
- In each of these sub-directories, I have two folders with name horses and humans, which contain the images which we want to use.

With just this, I now have a fully labelled dataset for training and testing. Because with tensorflow, I can pass these directories to something called a generator and it will auto-label these images based on directory name. This labelling is achieved using ```ImageDataGenerator``` in the ```keras``` library.

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

We can use this generator to apply transformations on the image. In this example we just apply normalization.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=128,
    class_mode='binary'
)

test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='binary'
)

In [7]:
# code for simple CNN that can classify horses and images

import tensorflow as tf

model = tf.keras.models.Sequential([
    # 9 layers, 3 (convolution, maxpool) pairs, 1 flatten, 2 dense
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',
                           input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 64)        0

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=0.001),
    metrics=['accuracy']
)

In [ ]:
# Training
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    steps_per_epoch=8,
    validation_steps=8.
    verbose=2
)

In [ ]:
import 